# ¶ 네이버 검색 API 활용한 이미지 데이터 수집

- 네이버 이미지 검색 결과를 출력해주는 REST API입니다. 
- 비로그인 오픈 API, GET으로 호출할 때 HTTP Header에 발급받은 Client ID와 Client Secret 값을 같이 전송해 주시면 활용 가능합니다.
* 네이버 검색 API 이미지 문서 : https://developers.naver.com/docs/serviceapi/search/image/image.md#%EC%9D%B4%EB%AF%B8%EC%A7%80
* ※ 컨텐츠 제작자 공개범위 등등 지정에 따라 네이버 통합 검색 결과와 일부 다르게 나올 수 있습니다.

In [1]:
import os
import sys
import pandas as pd
import json
import requests
import urllib.request
from urllib.parse import quote_plus
from logbook import Logger, StreamHandler
from tqdm import tqdm

StreamHandler(sys.stdout).push_application()
logger = Logger('NaverImageCrawling')

class NaverImageCrawling:
    """
        naver search api 활용한 이미지 수집
        
        naver_connect() : naver search api 호출
        request_count() : naver search api 요청 횟수
        search_start() : 검색어 목록에 따른 수집
        save_image(): 검색 결과 이미지 저장
    """
    
    def __init__(self, c_id, c_secret, dis=100, s_dir=None):
        """
            :param c_id: naver api client id
            :param c_secret: naver api client pwd
            :param dis: 한번에 수집할 건수 10(기본값), 100(최대)
            :param s_dir: 저장 폴더 경로
            from_num : 검색 결과 문서 중 문서의 시작점, 시작 위치 1(기본값), 1000(최대)
            i_count : 저장된 이미지 건수
        """           
        self.naver_client_id = c_id
        self.naver_client_secret = c_secret
        self.display = dis

        self.search_text = ''
        self.from_num = 1

        self.i_count = 0
        self.save_dir_path = s_dir

        if not self.save_dir_path:
            self.save_dir_path = os.path.dirname(os.path.abspath(os.getcwd())) + '/data/'

    def naver_connect(self):
        """
            naver search api 호출
            
            enc_text : 검색어 url encoding
            display : 한번에 수집할 건수 10(기본값), 100(최대)
            start : 검색 결과 문서 중 문서의 시작점, 시작 위치 1(기본값), 1000(최대)
            sort : 정렬 옵션: sim (유사도순), date (날짜순)
            :return resp: 요청 결과 (문자열)
        """              
        # 요청 변수
        enc_text = urllib.parse.quote(self.search_text) # naver search api utf-8 규격
        url = "https://openapi.naver.com/v1/search/image?query=" + enc_text + "&display=" + str(self.display) + "&start=" + str(self.from_num) + "&sort=date"
          
        # 요청 설정
        request = urllib.request.Request(url) # url지정하는 request 객체
        request.add_header("X-Naver-Client-Id", self.naver_client_id)
        request.add_header("X-Naver-Client-Secret", self.naver_client_secret)
        
        # 요청 응답
        response = urllib.request.urlopen(request) # 웹 서버에 정보를 요청한 후 응답받은 객체 저장.

        # 요청 에러
        rescode = response.getcode()
        
        resp = ''
        # 에러 처리 200(성공)
        if rescode == 200:
            response_body = response.read() # 반환된 객체의 byte배열로 읽기
            resp = response_body.decode("utf-8") # byte 배열을 문자열 변환
        else:
            logger.debug("naver_connect() Failed to response Error code:" + rescode)
            
        return resp

    def request_count(self):    
        """
            naver search api 요청 횟수
            요청 횟수: (총 검색된 이미지 건수/한번에 수집할 건수) + 1
            ex) (총 검색된 이미지 788건 /한번에 100건 수집) + 1 = 총 8번 요청
            
            :return range_end: api 요청 횟수
        """
        resp = self.naver_connect()
        range_end = 1
        
        if resp != '':
            image_dict = json.loads(resp)

            # 검색 결과가 1000개 이상 할때 
            range_end = 11
            total = image_dict['total']

            # 검색 결과가 1000개 이하일떄 총 검색 수를 변경하기 위함
            if total < 1000:

                # 기본 2부터 시작해야 함으로 + 1 더함
                range_end = int(total / self.display) + 1

                # 100단위 이하의 나머지를 처리하기 위함
                if total % self.display != 0:
                    range_end += 1

        return range_end

    def search_start(self, s_list, r_start=1):
        """
            검색어 목록에 따른 수집
            
            :param s_list: 검색어 목록
            :param r_start: 검색 시작 첫 위치
            i_count : 저장된 이미지 건수
            range_end : api 요청 횟수
            from_num : 검색 결과 문서의 시작점
        """       
        
        for search_text in tqdm(s_list):
            
            logger.info("Search word : %s" % search_text)
            
            self.i_count = 0 
            self.search_text = search_text
            
            # 한번에 수집할 건수 , 총 검색된 이미지 건수에 따른 api요청 횟수
            range_end = self.request_count()
            
            for s_idx, start in enumerate(range(r_start, range_end)):
        
                # 1이면 1부터, 2이면 101부터, 3=201, 4=301 ....부터 시작단위
                self.from_num = int(s_idx * self.display) + 1
                
                # 검색 결과 이미지 저장
                self.save_image(search_text)

            logger.info("Total save image %d \n" % self.i_count)

        logger.info("NaverImageCrawling end")
        

    def save_image(self, s_text):
        """
            검색 결과 이미지 저장
            
            :param s_text: search word
        """
        resp = self.naver_connect()
        
        if resp != '':
            image_dict = json.loads(resp)
            image_df = pd.DataFrame(image_dict['items'])

            save_dir = self.save_dir_path + s_text + '\\'

            if not os.path.isdir(save_dir):
                os.makedirs(save_dir)
            
            for idx, image_url in enumerate(image_df['link']):
                
                try:
                    # 이미지 주소를 다운로드를 위해 stream 모드로 가져온다.
                    session = requests.Session() # session 생성
                    r = session.get(image_url, stream=True) # http get request

                    # 에러 발생시 저장이 불가능하므로 건너뛰고 반복의 조건식으로 이동
                    if r.status_code != 200:
                        logger.debug("save_image() Failed to save image : (%d)" % r.status_code)
                        continue
                    
                    # 추출한 데이터를 저장
                    self.i_count += 1
                    file_path = save_dir + "%04d.jpg" % self.i_count
                    
                    # 'w': 텍스트 쓰기 모드, 'wb': 바이너리(이진값) 쓰기 모드
                    with open(file_path, 'wb') as f:
                        f.write(r.raw.read())
                        
                except Exception as ex:
                    logger.debug("save_image() Failed to image request: ", ex)
        else:
            logger.debug("Failed to save_image()")              
        
        
# naver search 실행 코드
if __name__ == '__main__':
    
    # application issued client id, pws
    client_id = "4QIyIbkYlmWcHZ8Uj1v0"
    client_secret = "bH730Lgcqn"
    
    # search word list 검색어를 지정합니다.
    search_list = ['마스크 쓴 사람', '마스크 사람']
    
    nic = NaverImageCrawling(client_id, client_secret) 
    nic.search_start(search_list)

  0%|                                                    | 0/2 [00:00<?, ?it/s]

[2021-06-21 05:59:29.960671] INFO: NaverImageCrawling: Search word : 마스크 쓴 사람
[2021-06-21 06:02:54.341295] DEBUG: NaverImageCrawling: save_image() Failed to save image : (404)
[2021-06-21 06:03:29.091468] INFO: NaverImageCrawling: Total save image 999 



 50%|█████████████████████▌                     | 1/2 [03:59<03:59, 239.13s/it]

[2021-06-21 06:03:29.095470] INFO: NaverImageCrawling: Search word : 마스크 사람
[2021-06-21 06:07:08.175744] INFO: NaverImageCrawling: Total save image 1000 



100%|███████████████████████████████████████████| 2/2 [07:38<00:00, 233.12s/it]


[2021-06-21 06:07:08.181748] INFO: NaverImageCrawling: NaverImageCrawling end
